In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import pandas as pd
import rasterio.plot

import overpy
import geopandas as gpd
# import ee
from shapely.geometry import Polygon, box
import os
# import geemap
import json
import multiprocessing



In [2]:
# os.chdir('/Users/yer1k/Documents/Master/Duke/2023 Fall/Capstone/solar_wastewater/30_result/')
os.chdir('/Users/yer1k/Library/CloudStorage/GoogleDrive-iamtsubasaozora@gmail.com/My Drive/Capstone Project/Data/')

In [3]:
api = overpy.Overpass()

In [23]:

# Fetch all wastewater treatment plants within California's boundary (# changed to Texas)
query = f"""
    area[admin_level=4]["name"="Florida"]->.searchArea; 
    (
    way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords

# plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}


geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df["centroid"] = df.to_crs('+proj=cea').centroid.to_crs(epsg=4326)



In [24]:
df

,WWTP_name,geometry,centroid
0,Plant_42335670,"POLYGON ((-82.68656 27.71888, -82.68566 27.718...",POINT (-82.68428 27.71757)
1,North District Wastewater Treatment Plant,"POLYGON ((-80.15290 25.92083, -80.15283 25.920...",POINT (-80.15027 25.91937)
2,Seas Backwash Basins,"POLYGON ((-81.55302 28.37564, -81.55292 28.375...",POINT (-81.55280 28.37562)
3,Plant_75199142,"POLYGON ((-82.78727 28.01833, -82.78586 28.018...",POINT (-82.78684 28.01985)
4,Pine Hills Water Treatment Plant,"POLYGON ((-81.45539 28.56399, -81.45627 28.563...",POINT (-81.45590 28.56545)
...,...,...,...
944,Plant_1237053617,"POLYGON ((-81.21605 28.62159, -81.21608 28.621...",POINT (-81.21662 28.62162)
945,Plant_1237058198,"POLYGON ((-81.28426 28.68673, -81.28608 28.687...",POINT (-81.28584 28.68695)
946,Plant_1237059631,"POLYGON ((-81.38361 28.82465, -81.38261 28.824...",POINT (-81.38249 28.82437)
947,Plant_1237059632,"POLYGON ((-81.38368 28.81960, -81.38290 28.819...",POINT (-81.38212 28.82104)


In [25]:
!ls

Archive                         Status Track.gsheet
PPT From Christine              WWTP Lists
Satellite Imagery_Automated RGB tif_bounding_box


In [26]:
# use loop to open the image in tif folder, and plot the bounding box on it
# read all the file names in the folder
for file in os.listdir('tif_bounding_box/Images of All 50 States/Florida'):
    try:
        with rasterio.open(os.path.join('tif_bounding_box/Images of All 50 States/Florida', file)) as src:
            # if file.startswith('download'):
                # continue
            # Read the image data
            img = src.read([1, 2, 3])  # Reads the first three bands for RGB
            # file name without extension
            name = os.path.splitext(file)[0]
            # find the corresponding bounding box
            df_name = df.loc[df['WWTP_name'] == name, :]
            # plot the bounding box
            fig, ax = plt.subplots(figsize=(8,8))
            # Plot the raster
            ax = rasterio.plot.show(img, ax=ax, transform=src.transform)
            # Plot the geometry on the ax axes instance
            df_name.boundary.plot(ax=ax, color='red', linewidth=1.5)
            plt.axis('off')
            # save the figure
            plt.savefig(os.path.join('tif_bounding_box/Images of All 50 States/Florida', name + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()
    except:
        print(f"fail to plot {file}")